In [14]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
import time
from tqdm import tqdm

In [15]:
from selenium import webdriver
#from selenium.webdriver.chrome.options import Options

#chrome_options = Options()
# headless 설정
#chrome_options.headless = True

C:\Users\jpc10\AppData\Local\Temp\ipykernel_27640\3192227794.py:6: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  chrome_options.headless = True


In [16]:
def musinsa_rank(category_num,page_num):
    url =  f"https://www.musinsa.com/ranking/best?period=now&age=ALL&mainCategory=00{category_num}&subCategory=&leafCategory=&price=&golf=false&kids=false&newProduct=false&exclusive=false&discount=false&soldOut=false&page={page_num}&viewType=small&priceMin=&priceMax="
    response = requests.get(url)
    html = bs(response.text, 'lxml')
    musinsa_rank_df = rbnl(html)

    return musinsa_rank_df

In [17]:
def rbnl(html):
    musinsa_rank_df = pd.DataFrame()

    #순위 뽑기
    rank_html = html.select('#goodsRankList > li > p')
    rank_no_list = []

    for i in rank_html:
        rank_no_list.append(i.string.strip())

    musinsa_rank_df['순위'] = rank_no_list

    #브랜드 이름 뽑기
    brand_html = html.select('#goodsRankList > li > div.li_inner > div.article_info > p.item_title > a')
    brand_list = []

    for i in brand_html:
        brand_list.append(i.string)



    #링크와 의류명 뽑기
    link_name_html = html.select('#goodsRankList > li > div.li_inner > div.article_info > p.list_info > a')
    link_list = []
    name_list = []

    for i in link_name_html:
        link_list.append(i['href'])
        name_list.append(i['title'])

    musinsa_rank_df['상품명']=name_list
    musinsa_rank_df['브랜드명']=brand_list
    musinsa_rank_df['url']=link_list

    #상세 페이지 크롤링
    musinsa_rank_df2 = specific_info(link_list)

    #데이터 프레임 옆으로 합치기
    musinsa_rank_df = pd.concat([musinsa_rank_df, musinsa_rank_df2], axis=1)

    return musinsa_rank_df

In [18]:
from pyasn1.compat.octets import null


def specific_info(link_list):
    headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.67 Safari/537.36'}

    musinsa_rank_df = pd.DataFrame()
    part_num_list=[]
    sex_list=[]
    view_list=[]
    sales_list=[]
    like_list=[]
    review_list=[]
    star_list=[]


    for link in tqdm(link_list):
        #         print(link)
        response_1=requests.get(link,headers=headers)
        html_1=bs(response_1.text, 'lxml')

        #품번 리스트 생성
        part_num_html=html_1.select('#product_order_info > div.explan_product.product_info_section > ul > li:nth-child(1) > p.product_article_contents > strong')
        part_num=part_num_html[0].get_text().split('/')[-1].strip()
        part_num_list.append(part_num)

        #성별 리스트 생성
        sex_html=html_1.select("#product_order_info > div.explan_product.product_info_section > ul > li > p.product_article_contents > span.txt_gender")
        sex=sex_html[0].get_text().replace('\n',' ').strip()
        sex_list.append(sex)

        #셀레니움으로 원하는 데이터 가져오기
        driver = webdriver.Chrome('chromedriver.exe')
        driver.get(link)
        sel_html=driver.page_source
        html_2=bs(sel_html)

        #조회수 가져오기
        view_html=html_2.find_all("strong", {"id":"pageview_1m"})
        view=view_html[0].get_text()
        view_list.append(view)

        #누적 판매 가져오기
        sales_html=html_2.find_all("strong", {"id":"sales_1y_qty"})
        sales=sales_html[0].get_text()
        sales_list.append(sales)

        #좋아요 수 가져오기
        like_html=html_2.find_all("span", {"class": "prd_like_cnt"})
        like=like_html[0].get_text()
        like_list.append(like)

        #평점
        star_html=html_2.find_all("span", {"class": "prd-score__rating" })
        if len(star_html) == 0:
            star = "0"
        else:
            star=star_html[0].get_text()
        star_list.append(star)


        #구매 후기 개수 가져오기
        review_html=html_2.find_all("span", {"class": "prd-score__review-count"})
        if len(review_html) == 0:
            review = "0"
        else:
            review=review_html[0].get_text()
        review_list.append(review)


        #드라이버 닫아주기
        driver.close()

        #시간 추가
        time.sleep(0.01)


    musinsa_rank_df['조회수']=view_list
    musinsa_rank_df['좋아요']=like_list
    musinsa_rank_df['구매후기 개수']=review_list
    musinsa_rank_df['평점']=star_list
    musinsa_rank_df['누적판매량(1년)']=sales_list


    return musinsa_rank_df

In [119]:
category_num=input("Category 번호를 입력하세요: ")
page_num=input("Page 번호를 입력하세요: ")
final_df=musinsa_rank(category_num,page_num)

0it [00:00, ?it/s]


In [116]:
file_name = f"0319 musinsa_ranking_category{category_num}_page{page_num}.csv"
final_df.to_csv(file_name, index=False)
pd.read_csv(file_name)

,순위,상품명,브랜드명,url,조회수,좋아요,구매후기 개수,평점,누적판매량(1년)
0,4411위,Color Line Knit (Brown),아워스코프,https://www.musinsa.com/app/goods/1155740?loc=...,400회 이상,126,후기 5개 보기,5,NaN
1,4412위,RETRIEVER FRIENDS CREWNECK [MELANGE GRAY],리트리버클럽,https://www.musinsa.com/app/goods/2140537?loc=...,8.3천 회 이상,"13,233","후기 1,428개 보기",4.9,3.2천 개 이상
2,4413위,엠블럼 져지 롱슬리브 티셔츠_블랙,오아이오아이,https://www.musinsa.com/app/goods/3149903?loc=...,700회 이상,125,후기 0개,평점 0점,NaN
3,4414위,[패키지] 아크로매틱 플루이드 린넨 포켓 셔츠,브렌슨,https://www.musinsa.com/app/goods/2578564?loc=...,5.3천 회 이상,"6,621",후기 802개 보기,4.8,4.1천 개 이상
4,4415위,메쉬드 반팔 카라 스웨터 (베이지),엠엠지엘,https://www.musinsa.com/app/goods/2469400?loc=...,700회 이상,947,후기 66개 보기,4.8,100개 이상
...,...,...,...,...,...,...,...,...,...
85,4496위,벌룬핏 크루넥 니트 BLACK,인사일런스,https://www.musinsa.com/app/goods/2780269?loc=...,2.2천 회 이상,351,후기 46개 보기,4.9,100개 이상
86,4497위,Narbe Half Shirt - Blue Grey,오프닝프로젝트,https://www.musinsa.com/app/goods/3051777?loc=...,1.4천 회 이상,173,후기 0개,평점 0점,NaN
87,4498위,PHYPS® MONDAY ROUTINE STITCH HOODIE BERRY PINK,피지컬 에듀케이션 디파트먼트,https://www.musinsa.com/app/goods/3073854?loc=...,6.2천 회 이상,924,후기 24개 보기,4.5,100개 이상
88,4499위,차콜그레이 벌룬핏 피그먼트 다잉 후드 티셔츠 IETS3E201CG,일꼬르소,https://www.musinsa.com/app/goods/2873711?loc=...,4.4천 회 이상,802,후기 61개 보기,5,200개 이상
